In [ ]:
import pandas as pd
from config import Config
import numpy as np

In [ ]:
cnfg = Config()

In [ ]:
external_1 = pd.read_csv(cnfg.external_1)

In [ ]:
external_2 = pd.read_csv(cnfg.external_2)

In [ ]:
external_1.head()

,problem,level,type,solution,stage,source
0,Kevin Kangaroo begins hopping on a number line...,Level 5,Algebra,Kevin hops $1/3$ of the remaining distance wit...,train,MATH
1,The ratio of the areas of two squares is $\fra...,Level 4,Algebra,We start off by simplifying the ratio $\frac{1...,train,MATH
2,"If $\sqrt{2\sqrt{t-2}} = \sqrt[4]{7 - t}$, the...",Level 4,Algebra,"We raise both sides to the fourth power, which...",train,MATH
3,Let $t(x) = \sqrt{3x+1}$ and $f(x)=5-t(x)$. Wh...,Level 4,Algebra,We first evaluate $f(5) = 5 -t(5) = 5-\sqrt{5\...,train,MATH
4,James has a total of 66 dollars in his piggy b...,Level 2,Algebra,Call the number of one dollar bills $x$ and th...,train,MATH


In [ ]:
external_2.head()

,problem_id,link,problem,solution,letter,answer
0,4ba30954e5f3ca72748b3e145f45b705,https://artofproblemsolving.com/wiki/index.php...,"What is the ones digit of \[222,222-22,222-2,2...","We can rewrite the expression as \[222,222-(22...",B,2
1,4ba30954e5f3ca72748b3e145f45b705,https://artofproblemsolving.com/wiki/index.php...,"What is the ones digit of \[222,222-22,222-2,2...","222,222-22,222 = 200,000\n200,000 - 2,222 = 19...",B,2
2,4ba30954e5f3ca72748b3e145f45b705,https://artofproblemsolving.com/wiki/index.php...,"What is the ones digit of \[222,222-22,222-2,2...","We only care about the unit's digits.\nThus, $...",B,2
3,4ba30954e5f3ca72748b3e145f45b705,https://artofproblemsolving.com/wiki/index.php...,"What is the ones digit of \[222,222-22,222-2,2...",We just take the units digit of each and subtr...,B,2
4,085955dda8dfb374689b3f216b54d785,https://artofproblemsolving.com/wiki/index.php...,What is the value of this expression in decima...,We see that $\frac{44}{11}$ is $4$ $\frac{110}...,C,6.54


In [ ]:
external_comb = pd.read_csv(cnfg.ext_comb_pre_processed)

In [ ]:
external_comb.shape

(27861, 10)

In [ ]:
external_comb.head()

,problem_id,problem,solution,answer,link,source,level,type,stage,letter
0,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...","We can rewrite the expression as \[222,222-(22...",2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
1,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...","222,222-22,222 = 200,000\n200,000 - 2,222 = 19...",2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
2,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...","We only care about the unit's digits.\nThus, $...",2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
3,4ba30954e5f3ca72748b3e145f45b705,"What is the ones digit of \[222,222-22,222-2,2...",We just take the units digit of each and subtr...,2.00,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,B
4,085955dda8dfb374689b3f216b54d785,What is the value of this expression in decima...,We see that $\frac{44}{11}$ is $4$ $\frac{110}...,6.54,https://artofproblemsolving.com/wiki/index.php...,AOPS,NaN,NaN,train,C


In [ ]:
def remove_multiple_choice_options_in_problem(problem_str):
    return problem_str.rsplit("$\\textbf{(A", 1)[0].rsplit("$\\text{(A", 1)[0].strip()


def remove_asy_block(text):
    if text.count("[asy]") > 0:
        text = text.split("[asy]", 1)[0] + text.split("[/asy]", 1)[-1]
        remove_asy_block(text)
    return text.strip()


def fix_and_filter_external_data(df: pd.DataFrame, drop_non_pos_int_rows: bool = True):
    """"""
    _df = df.copy()

    # Drop rows with infinite values in answer
    _df = _df[~_df["answer"].replace([np.inf, -np.inf], np.nan).isna()]

    # Force answer to be appropriate dtype (int or float)
    _df["answer"] = _df["answer"].apply(
        lambda x: int(x) if float(int(x)) == float(x) else float(x)
    )

    # Remove multiple choices in problem string
    _df.loc[~pd.isna(_df["letter"]), "problem"] = _df.loc[
        ~pd.isna(_df["letter"]), "problem"
    ].apply(remove_multiple_choice_options_in_problem)

    # Replace hashtag solutions with boxed
    _df.loc[_df.solution.str.contains("#### "), "solution"] = _df.loc[
        _df.solution.str.contains("#### "), "solution"
    ].apply(
        lambda x: x.strip().rsplit("####", 1)[0]
        + "$\\boxed{"
        + x.strip().rsplit("####", 1)[-1].strip()
        + "}$"
    )

    # Drop rows where the answer is less than 0 or non-int
    if drop_non_pos_int_rows:
        _df = _df[_df.answer.apply(lambda x: isinstance(x, int) and 0 <= x)]

    # Drop rows where there is

    # Modulo the answer and store the original in a separate column
    _df["original_answer"] = _df["answer"].copy()
    _df["answer"] = _df["answer"].apply(lambda x: x % 1000)

    return _df.reset_index(drop=True)


def filter_by_consensus(
    df: pd.DataFrame, id_col: str = "problem_id", answer_col: str = "answer"
) -> pd.DataFrame:
    """
    Filters a DataFrame to retain only rows where the answer has the majority consensus
    for each unique problem identifier.

    This function groups the DataFrame by a problem identifier and determines the most frequent
    answer for each group. Only rows where the answer matches the most frequent (mode) answer
    for their corresponding group are retained.

    Args:
        df (pd.DataFrame): The DataFrame to filter.
        id_col (str, optional): The column name in the DataFrame that contains the problem identifiers.
                                Defaults to 'problem_id'.
        answer_col (str, optional): The column name in the DataFrame that contains the answers.
                                    Defaults to 'answer'.

    Returns:
        pd.DataFrame: A DataFrame containing only the rows with the majority consensus answer
                      for each unique problem identifier.

    Example:
        >>> data = {'problem_id': ['1', '1', '1', '2', '2', '3'],
        ...         'answer': [10, 10, 4, 150, 150, 3]}
        >>> df = pd.DataFrame(data)
        >>> filtered_df = filter_by_consensus(df)
        >>> print(filtered_df)
    """
    _df = df.copy()

    # Calculate the mode of the answers for each problem_id
    mode_df = (
        _df.groupby(id_col)[answer_col]
        .agg(lambda x: pd.Series.mode(x)[0])
        .reset_index()
    )
    mode_df.rename(columns={answer_col: "mode_answer"}, inplace=True)

    # Merge this back with the original DataFrame to filter
    merged_df = _df.merge(mode_df, on=id_col)

    # Keep only rows where the answer matches the mode answer
    result_df = merged_df[merged_df[answer_col] == merged_df["mode_answer"]]

    # Remove the temporary mode_answer column
    result_df = result_df.drop(columns=["mode_answer"])

    return result_df.reset_index(drop=True)

In [ ]:
external_comb.shape

(27861, 10)

In [ ]:
external_comb = fix_and_filter_external_data(external_comb)

In [ ]:
external_comb.shape

(24952, 11)

In [ ]:
external_comb = filter_by_consensus(external_comb)

In [ ]:
external_comb.shape

(24943, 11)

In [ ]:
external_comb.iloc[0].solution

'We can rewrite the expression as \\[222,222-(22,222+2,222+222+22+2).\\]\nWe note that the units digit of the addition is $0$ because all the units digits of the five numbers are $2$ and $5*2=10$ , which has a units digit of $0$\nNow, we have something with a units digit of $0$ subtracted from $222,222$ . The units digit of this expression is obviously $2$ , and we get $\\boxed{2}$ as our answer.'

In [ ]:
external_comb.to_csv(f'{cnfg.base_data_path}aimo_external/cleaned_comb.csv', index=False)